In [4]:
import random
import string
import numpy as np
random.seed()

# Import text from the file

In [5]:
f = open('edgar_allan_poe.txt')
text = f.read()

# Format text into list of lines, lowercase and punctuation removed

In [6]:
lines_formatted = []
lines = text.splitlines()
for line in lines:
    if line:
        lines_formatted.append(line.lower().translate(str.maketrans('', '', string.punctuation)))

# Iterate through lines to populate pi and A dictionaries:

Pi is a dictionary where the keys are the words and the values are the probabilities.

We will need two A dictionaries, one for first order probabilities, for the second word in a line, and one for second order probabilities, for the rest of the line.

A1 is a dictionary where the keys are the words and the values are further dictionaries, containing probabilities for following words.

A2 is a dictionary where the keys are word pairs and the values are further dictionaries as with A1.

Loop through each line in all input text, and in each line loop through each word:


*   If the word is in the first position in the line, check if it exists in Pi dictionary. If so, increment its value by 1/N where N is number of lines. If not, add it to the dictionary with value 1/N.
*   If the word is in the second position:
*   Check if the previous word exists in A1 dictionary. If so, check if current word exists in subdictionary. If so, increment its value by 1. If current word does not exist in subdictionary, add it with value 1. If previous word does not exist in A1 dictionary, add it and add current word to subdictionary with value 1.
*   If the word is in any other position:
*   Check if the previous word-pair exists in A2 dictionary. If so, check if current word exists in subdictionary.... as above
*   For each word in A1 and word-pair in A2 dictionaries, sum up total of values in subdictionary, and divide each number by that count, such that each one sums to 1.








In [7]:
from re import A
pi = {}
A1 = {}
A2 = {}
words = []
#sum = 0

for line in lines_formatted:
    words = line.split(' ')
    if '' in words:
        words.remove('')
    for i in range(len(words)):
        if i == 0: #if this is the first word in the line (update pi)
            if words[i] in pi: #if it exists in the pi dictionary
                pi[words[i]] += 1/len(lines_formatted)
            else: #if it does not exist in the pi dictionary
                pi.update({words[i]: 1/len(lines_formatted)})
        elif i == 1: #if this is the second word in the line (update A1)
            if words[i-1] in A1: #if the previous word exists in the A1 dictionary
                if words[i] in A1[words[i-1]]: #if the current word exists in the previous word's subdictionary
                    A1[words[i-1]][words[i]] += 1
                else: #if the current word does not exist in the subdictionary
                    A1[words[i-1]].update({words[i]: 1})
            else:
                A1.update({words[i-1]: {words[i]: 1}})
        else: #if this is not the first or second word in the line (update A2)
            if (words[i-2],words[i-1]) in A2:
                if words[i] in A2[(words[i-2],words[i-1])]:
                    A2[(words[i-2],words[i-1])][words[i]] += 1
                else:
                    A2[(words[i-2],words[i-1])].update({words[i]: 1})
            else:
                A2.update({(words[i-2],words[i-1]): {words[i]: 1}})

for i in A1:
    word_count = sum(A1[i].values())
    for j in A1[i]:
        A1[i][j] = A1[i][j]/word_count

for i in A2:
    word_count = sum(A2[i].values())
    for j in A2[i]:
        A2[i][j] = A2[i][j]/word_count

# Generate Text using pi and A

Now generate a poem. For each line, generate a first word using pi, a second word using A2, then generate words following it using A2. If encountering a word which doesn't exist in A2 (that is, it only exists in the training text as the end of a line) then add a full stop and stop generating.

generateLine and generatePoem will be functions allowing variable line length and line number.

In [8]:
def generateLine(line_length):
    random_line = []

    #generate first word using pi
    words = list(pi.keys())
    weights = list(pi.values())
    random_line.append(random.choices(words,weights)[0])

    #generate second work using A1
    words = list(A1[random_line[0]].keys())
    weights = list(A1[random_line[0]].values())
    random_line.append(random.choices(words,weights)[0])

    for i in range(line_length-2):
        if (random_line[i],random_line[i+1]) not in A2:
            random_line.append('.')
            break
        words = list(A2[(random_line[i],random_line[i+1])].keys())
        weights = list(A2[(random_line[i],random_line[i+1])].values())
        random_line.append(random.choices(words,weights)[0])
    return " ".join(random_line).replace(' .','.')

In [9]:
def generatePoem(line_length, line_number):
    random_poem = []
    for _ in range(line_number):
        random_poem.append(generateLine(line_length))
    return "\n".join(random_poem)

In [11]:
print(generatePoem(16,4))

that i journeyed down here .
of which it doth now know.
admiring natures universal throne.
and the giddy stars so legends tell.
